In [1]:
from os.path import join
import numpy as np
import pandas as pd

from feature.dataset import make_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from category_encoders import *

train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')
holiday_path = join('jeju_data', '국가공휴일.csv')

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

x_train, y_train, test = make_dataset(train_path, test_path, holiday_path)

Start time:  2022-11-03 15:48:50.420807
Train dataset success !
Test dataset success !
End time: 2022-11-03 15:50:46.059236
Play time:  0:01:55.638429


In [41]:
X = x_train.copy()
y = y_train.copy()
df = pd.concat([X, y], axis=1)

In [67]:
X.columns

Index(['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
       'multi_linked', 'connect_code', 'maximum_speed_limit',
       'weight_restricted', 'road_type', 'start_node_name', 'start_latitude',
       'start_longitude', 'start_turn_restricted', 'end_node_name',
       'end_latitude', 'end_longitude', 'end_turn_restricted', 'sin_time',
       'cos_time', 'group_time', 'month', 'week', 'post_holiday',
       'pre_holiday', 'holiday', 'season', 'vacation', 'distance', 'jeju_dist',
       'seogwi_dist', 'node_TF', 'diff', 'road_name_set', 'section_speed',
       'start_speed', 'end_speed', 'turn_restricted', 'rest',
       'location_cluster'],
      dtype='object')

In [64]:
X

,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,...,seogwi_dist,node_TF,diff,road_name_set,section_speed,start_speed,end_speed,turn_restricted,rest,location_cluster
0,1,17,1,106,6,0,0,60.0,32400.0,3,...,21.516988,True,17.683713,1,49.105982,48.697943,50.298219,0,0.0,3
1,1,21,2,103,30,0,0,60.0,0.0,0,...,28.052074,False,17.683713,1,47.203323,26.400712,26.400712,1,0.0,0
2,4,7,2,103,32,0,0,80.0,0.0,0,...,18.626831,False,19.296304,1,56.858438,59.101720,65.118140,0,1.0,2
3,0,13,2,107,52,0,0,50.0,0.0,0,...,1.110065,False,14.507769,2,25.030004,23.755158,25.445418,0,0.0,1
4,6,8,2,103,31,0,0,80.0,0.0,0,...,31.524609,False,19.296304,1,51.188650,39.873670,39.873670,0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,1,16,1,107,0,0,0,50.0,0.0,0,...,32.429156,False,14.507769,0,37.755208,27.620482,28.384300,0,0.0,2
4701213,1,2,2,107,6,0,0,80.0,43200.0,3,...,27.338748,True,19.296304,1,64.204001,65.385516,68.900028,0,0.0,0
4701214,3,22,2,103,31,0,0,60.0,0.0,0,...,38.995594,False,17.683713,1,42.717892,33.398433,41.850580,0,0.0,3
4701215,2,2,2,103,33,0,0,80.0,0.0,0,...,24.270769,False,19.296304,1,69.543011,61.554919,59.264147,0,0.0,0


In [62]:
test

,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,...,seogwi_dist,node_TF,diff,road_name_set,section_speed,start_speed,end_speed,turn_restricted,rest,location_cluster
0,1,17,3,107,29,0,0,70.0,0.0,0,...,27.467909,False,28.633792,2,35.053946,33.623164,33.623164,1,0.0,0
1,6,12,2,103,6,0,0,70.0,0.0,3,...,13.419111,False,28.633792,1,42.798824,49.240040,48.359276,0,0.0,1
2,0,2,1,103,32,0,0,60.0,0.0,0,...,7.959867,False,17.683713,1,42.085292,58.979119,58.979119,0,0.0,1
3,1,23,3,103,30,0,0,70.0,0.0,0,...,24.160820,False,28.633792,1,26.064089,33.185444,29.584532,0,0.0,0
4,2,17,3,106,11,0,0,70.0,0.0,0,...,27.063194,False,28.633792,2,52.053757,46.861501,46.619422,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,5,5,1,103,32,0,0,80.0,0.0,0,...,25.763104,False,19.296304,1,56.858438,39.550643,42.992907,0,1.0,0
291237,0,20,2,103,30,0,0,60.0,0.0,0,...,6.398556,False,17.683713,1,47.203323,54.630245,55.011294,0,0.0,1
291238,0,11,1,107,22,0,0,30.0,0.0,0,...,27.690209,False,-2.220825,2,29.095771,29.095771,29.095771,2,0.0,0
291239,0,7,2,107,1,0,0,60.0,0.0,0,...,4.907302,False,17.683713,2,24.686785,23.236410,24.378022,0,0.0,1


In [45]:
from category_encoders import *
cat_col = ['road_name', 'start_node_name', 'end_node_name']
cbe = CatBoostEncoder(cols=cat_col)
X1 = cbe.fit_transform(X, y)
test1 = cbe.transform(test)

In [55]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler


def objective_xgb(trial: Trial, x, y):
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_int('gamma', 1, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'random_state': 42
    }

    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

    model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0)
    xgb_model = model.fit(x_train, y_train, verbose=False, eval_set=[(x_val, y_val)], early_stopping_rounds=50)
    y_pred = xgb_model.predict(x_val)
    score = mean_absolute_error(y_val, y_pred)

    return score


study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=30)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

params = study.best_trial.params
xgb_model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0).fit(x_train, y_train)
y_pred = xgb_model.predict(test)

import pandas as pd

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_xgb_optunadelout.csv", index=False)

[I 2022-11-03 18:56:21,789] A new study created in memory with name: no-name-c39c666d-db21-43cc-9170-bdc2a958fca8
C:\Users\jkpark\AppData\Local\Temp\ipykernel_8268\3082868993.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_8268\3082868993.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
[I 2022-11-03 19:05:19,649] Trial 0 finished with value: 3.0706219099619787 and parameters: {'n_estimators': 1307, 'max_depth': 14, 'min_child_weight': 160, 'gamma': 3, 'learning_rate': 0.008, 'co

Best trial: score 2.972929936050585,
params {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018, 'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}


In [56]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


param = study.best_trial.params

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      XGB = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0)
      XGB.fit(x_train, y_train)
      
      y_pred = XGB.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      XGB_model[f] = XGB
      print(f'================================================================================\n\n')
              


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5    
    


c:\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.955053659996513


====================================2============================================
2 Fold MAE = 2.9566476237456882


====================================3============================================
3 Fold MAE = 2.9533176913849553


====================================4============================================
4 Fold MAE = 2.9544595097355377


====================================5============================================
5 Fold MAE = 2.954298525928119




In [59]:
param = study.best_trial.params
param

{'n_estimators': 4693,
 'max_depth': 15,
 'min_child_weight': 4,
 'gamma': 1,
 'learning_rate': 0.018,
 'colsample_bytree': 0.9015018647603987,
 'lambda': 1.386941960727803,
 'alpha': 0.10534800837686535,
 'subsample': 1.0}

In [57]:
sample_submission.to_csv("./submit_xgb_fold.csv", index=False)

In [58]:
df_imp = pd.DataFrame({'imp':XGB.feature_importances_}, index = XGB.feature_names_in_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
day_of_week,0.000421
post_holiday,0.000443
holiday,0.000612
vacation,0.000614
pre_holiday,0.000620
rest,0.000745
month,0.000781
multi_linked,0.000888
week,0.000993
sin_time,0.001363
